In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45333
2,Huelva,Confirmados PDIA 14 días,335
3,Huelva,Tasa PDIA 14 días,"65,28051133152756"
4,Huelva,Confirmados PDIA 7 días,141
5,Huelva,Tasa PDIA 7 dias,"27,476274918642943"
6,Huelva,Total Confirmados,45536
7,Huelva,Curados,44235
8,Huelva,Fallecidos,414


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45333.0


/tmp/ipykernel_4417/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12528.0


/tmp/ipykernel_4417/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_4417/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_4417/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_4417/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4960 personas en los últimos 7 días 

Un positivo PCR cada 2055 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45333.0,141.0,335.0,513170.0,27.476275,65.280511,41.0
Huelva-Costa,26841.0,90.0,202.0,289548.0,31.082929,69.763908,25.0
Condado-Campiña,14071.0,44.0,117.0,156231.0,28.163425,74.889107,12.0
Huelva (capital),12528.0,29.0,70.0,143837.0,20.161711,48.666199,9.0
Isla Cristina,3123.0,13.0,23.0,21393.0,60.767541,107.511803,6.0
Gibraleón,936.0,6.0,13.0,12737.0,47.106854,102.064850,4.0
Sierra de Huelva-Andévalo Central,3997.0,7.0,16.0,67391.0,10.387144,23.742043,4.0
Ayamonte,2070.0,12.0,34.0,21104.0,56.861259,161.106899,3.0
Bollullos Par del Condado,1280.0,7.0,21.0,14387.0,48.655036,145.965107,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Higuera de la Sierra,70.0,1.0,5.0,1291.0,77.459334,387.296669,0.0
Paterna del Campo,207.0,1.0,12.0,3457.0,28.926815,347.121782,0.0
Ayamonte,2070.0,12.0,34.0,21104.0,56.861259,161.106899,3.0
Bollullos Par del Condado,1280.0,7.0,21.0,14387.0,48.655036,145.965107,3.0
Palos de la Frontera,1027.0,10.0,17.0,11742.0,85.164367,144.779424,0.0
Cartaya,1969.0,14.0,29.0,20083.0,69.710701,144.400737,2.0
Isla Cristina,3123.0,13.0,23.0,21393.0,60.767541,107.511803,6.0
Gibraleón,936.0,6.0,13.0,12737.0,47.106854,102.064850,4.0
Almonte,2284.0,11.0,23.0,24507.0,44.885135,93.850737,2.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paterna del Campo,207.0,1.0,12.0,3457.0,28.926815,347.121782,0.0,0.083333
Higuera de la Sierra,70.0,1.0,5.0,1291.0,77.459334,387.296669,0.0,0.200000
Moguer,1953.0,5.0,18.0,21867.0,22.865505,82.315818,2.0,0.277778
Bollullos Par del Condado,1280.0,7.0,21.0,14387.0,48.655036,145.965107,3.0,0.333333
Bonares,458.0,1.0,3.0,6060.0,16.501650,49.504950,0.0,0.333333
Ayamonte,2070.0,12.0,34.0,21104.0,56.861259,161.106899,3.0,0.352941
Condado-Campiña,14071.0,44.0,117.0,156231.0,28.163425,74.889107,12.0,0.376068
Lepe,3007.0,9.0,22.0,27880.0,32.281205,78.909613,3.0,0.409091
Huelva (capital),12528.0,29.0,70.0,143837.0,20.161711,48.666199,9.0,0.414286
